In [1]:
pip install git+https://github.com/huggingface/transformers accelerate

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-hdfetzgu
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-hdfetzgu
  Resolved https://github.com/huggingface/transformers to commit 62db3e6ed67a74cc1ed1436acd9973915c0a4475
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached tokenizers-0.21.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached tokenizers-0.21.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)
  Created wheel for transformers: filename=transformers-4.49.0.dev0-py3-none-any.whl size=10627237 sha256=07def7f3cd52ae9c9f11a57abb49b25c3483adc9cb0336f6e4977db39bfe65cd
  Stored in directory: /tmp/pip-ephem-wheel-cache-qz6k5th0/wheels/14/a0/7b/8f6b25ba4110aa215fcb8d6aedd6cd4f9b9b6619190999ac2b
Successfully built transformers
  Attempting 

In [2]:
pip install qwen-vl-utils[decord]==0.0.8

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 68.5 MB/s eta 0:00:00:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
import torchvision
import transformers
import torch

/home/do.ng/.conda/envs/CS7980_YouDescribe/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
from transformers import Qwen2_5_VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info

model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2.5-VL-7B-Instruct", torch_dtype="auto", device_map="auto"
)
processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-7B-Instruct")

Loading checkpoint shards: 100%|██████████| 5/5 [00:00<00:00,  5.38it/s]


In [16]:
import json
def get_scene_data(scene_number, scenes_json_path):
    with open(scenes_json_path, 'r') as f:
        scenes = json.load(f)

    for scene in scenes:
        if scene["scene_number"] == scene_number:
            return scene

    raise ValueError(f"Scene {scene_number} not found in {scenes_json_path}")

In [20]:
scene_4_data = get_scene_data(4, f"videos/rlOywSJnPOU/rlOywSJnPOU_scenes/scene_info.json")

In [21]:
scene_4_data

{'scene_number': 4,
 'start_frame': 1776,
 'end_frame': 3314,
 'start_time': 59.25894863717299,
 'end_time': 110.57666429256267,
 'duration': 51.31771565538968,
 'video_path': 'videos/rlOywSJnPOU/rlOywSJnPOU_scenes/scene_004.mp4',
 'captions': ['150g tuna',
  'Filter the oil',
  '35g (big 2tbsp) chopped onion, chopped',
  '35g (big 2tbsp) chopped pickles',
  '35g (big 2tbsp) chopped celery',
  '60g (big 3tbsp) mayonnaise large',
  '2g (0.7tbsp) crushed red pepper',
  '1 tbsp of lemon juice',
  '0.5tsp black pepper',
  'Mix well']}

In [26]:
def query_video(prompt, scene_data):
    torch.cuda.empty_cache()

    video_length = scene_data.get("duration")

    fps = 1.0
    max_frames_value = min(32, int(video_length * fps)) 
    
    if scene_data["captions"]:
        captions_text = "\n".join([f"- {caption}" for caption in scene_data["captions"]])
        scene_context = f"Scene {scene_data['scene_number']} captions:\n{captions_text}"
    
    enhanced_prompt = f"{scene_context}\n{prompt}"

    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "video",
                    "video": scene_data['video_path'],
                    "max_pixels": 320 * 240,
                    "fps": 1.0,
                    "max_frames": max_frames_value,
                },
                {"type": "text", "text": enhanced_prompt},
            ],
        }
    ]

    print(f"Processing Scene {scene_data['scene_number']}")

    dtype = torch.bfloat16

    with torch.amp.autocast('cuda', dtype=dtype):
        text = processor.apply_chat_template(
            messages, tokenize=False, add_generation_prompt=True
        )
        image_inputs, video_inputs = process_vision_info(messages)
        inputs = processor(
            text=[text],
            images=image_inputs,
            videos=video_inputs,
            padding=True,
            return_tensors="pt",
        )

        inputs = inputs.to("cuda", dtype=dtype)
        input_length = inputs.input_ids.shape[1]

        with torch.no_grad():
            generated_ids = model.generate(
                **inputs,
                max_new_tokens=128,
                do_sample=False,
                num_beams=1
            )

        generated_ids_trimmed = generated_ids[:, input_length:]

        output_text = processor.batch_decode(
            generated_ids_trimmed,
            skip_special_tokens=True,
            clean_up_tokenization_spaces=False
        )

    del inputs, generated_ids, generated_ids_trimmed
    torch.cuda.empty_cache()

    return output_text

In [27]:
query_video("describe the video in detail", scene_data=scene_4_data)

Processing Scene 4


/home/do.ng/.conda/envs/CS7980_YouDescribe/lib/python3.9/site-packages/torch/amp/autocast_mode.py:266: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
qwen-vl-utils using decord to read video.


RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx